<a href="https://colab.research.google.com/github/LucioFassarella/QCOP/blob/main/QCOP_hamiltoniano.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# QISKIT: Carregamento

In [1]:
!pip install qiskit -U
!pip install qiskit_aer -U
!pip install qiskit-ibm-runtime -U

!pip install matplotlib
!pip install pylatexenc

import qiskit
qiskit.__version__

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.9/363.9 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.3
    Uninstalling requests-2.32.3:
      Successfully uninstalled requests-2.32.3
ERROR: pip's dependency resolver does not currently

'2.1.0'

In [3]:
# Qiskit: métodos básicos

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister

from qiskit import transpile

from qiskit.visualization import plot_histogram, array_to_latex, plot_state_city

# Problemas de Otimização Combinatória: Hamiltoniano

Espaço das sequências binárias com $n \in \mathbb{N}$ termos:

$$
\mathcal{Z}(n) = \left\{(z_{0}, \dots, z_{n-1});\ z_0, \dots, z_{n-1} \in \left\{0,1\right\}\right\}.
$$

Função objetivo:

$$
C:S \rightarrow \mathbb{R},\ \ S \subseteq \mathcal{Z}(n).
$$

<!-- Para $z = (z_0,\dots, z_{n-1}) \in \mathcal{Z}(n)$ vale:

\begin{equation*}
	\begin{split}
		|z\rangle \langle z | &= |z_{n-1}\rangle \langle z_{n-1} |\otimes \dots \otimes |z_0\rangle \langle z_0 |\\
		&= \frac{I + (-1)^{z_{n-1}}Z_{n-1}}{2}\otimes \dots \otimes \frac{I + (-1)^{z_0}Z_{0}}{2}\\
		&= I + \sum_{k=1}^n\sum_{0 \le j_1 < \dots <j_k \le n-1}%
		(-1)^{z_{j_1} + \dots + z_{j_k}}Z_{j_1} \dots Z_{j_k}.
	\end{split}
\end{equation*}
-->

Hamiltoniano em termos dos operadores de Pauli é dada por:

\begin{equation}
	\begin{split}
		H_C &= \sum_{z \in S}C(z)|z\rangle \langle z |\\
		&= \frac{1}{2^n}\sum_{z \in S}C(z)%
		\left\lbrack I + \sum_{k=0}^{n-1}\sum_{0 \le j_0 < \dots <j_k \le n-1}%
		(-1)^{z_{j_1} + \dots + z_{j_k}}Z_{j_1} \dots Z_{j_k}
		\right\rbrack.
	\end{split}
\end{equation}

In [39]:
def hamiltoniano(n = "int", C = "function", S = "None"):
    '''
    Função que constrói o Hamiltoniano em termos dos operadores de Pauli.

    Entrada:
        n: tipo = inteiro: --> número de termos.
        C: tipo = função: --> função objetivo.
        S: tipo = lista: --> subconjunto das sequências binárias com n termos.

    Saída:
        H_C: tipo = lista --> Hamiltoniano em termos dos operadores de Pauli.

    Métodos:
        copy(): < https://docs.python.org/3/library/copy.html >
        SparsePauliOp: < https://quantum.cloud.ibm.com/docs/en/api/qiskit/qiskit.quantum_info.SparsePauliOp >

    Funções:
        sequencias_binarias(n = "int"): --> lista de todas as sequências binárias com n termos.
    '''

    # Métodos

    from qiskit.quantum_info import SparsePauliOp
    from copy import deepcopy as dcopy

    # Condições

    # Construção de HC

    if S == "None":
        sequencias = sequencias_binarias(n)
    else:
        sequencias = dcopy(S)

    for sequencia in sequencias:
        sequencia.reverse()

    indices = subsequencias(range(n))

    operadores = []
    # op = SparsePauliOp.from_sparse_list([("ZX", [1, 4], 1), ("YY", [0, 3], 2)], num_qubits=5)

    for sequencia in sequencias:
        coeficiente = C(sequencia)/2**n

        if coeficiente == 0:
            pass

        else:
            operadores.append(("I", [0] , coeficiente))

            for indice in indices:
                sinal = (-1)**sum(sequencia[j] for j in indice)
                operadores.append((len(sequencia)*"Z", indice , sinal*coeficiente))

    H_C = SparsePauliOp.from_sparse_list(operadores, num_qubits=n)

    return H_C

In [40]:
def sequencias_binarias(n = "int"):
    '''
    Função que contrói o conjunto das sequências binárias com n termos.

    Entrada:
        n: tipo = inteiro: --> número de termos.

    Saída:
        Z_n: tipo = lista --> sequências binárias com n termos.

    Métodos:
        copy(): < https://docs.python.org/3/library/copy.html >
    '''
    # Métodos

    from copy import deepcopy as dcopy

    # Condição

    if type(n) != int or n <= 0:
        print("ERRO: A entrada deve ser um inteiro positivo.")
        return []

    # Loop


    else:
        if n == 1:
            Z_n = [[0], [1]]

        else:
            sequencias = sequencias_binarias(n - 1)
            sequencias_0 = dcopy(sequencias)
            sequencias_1 = dcopy(sequencias)
            for sequencia in sequencias_0:
                    sequencia.insert(0,0)
            for sequencia in sequencias_1:
                    sequencia.insert(0,1)
            Z_n = sequencias_0 + sequencias_1
        return Z_n

In [45]:
def subsequencias(sequencia):
    '''
    Função que constrói todas as subsequências de uma dada sequência.

    Entrada:
        sequencia: tipo = list --> sequência

    Saída:
        lista_subsequencias = list --> lista de todas as subsequências da sequência.

    Métodos:
        intertools.combinations(): < https://docs.python.org/3/library/itertools.html >

    '''
    # Método

    from itertools import combinations

    # Condições:

    if type(sequencia) != list:
        print("ERRO: A entrada deve ser uma lista.")
        return []

    # Loop:

    sublistas = []
    for r in range(len(sequencia) + 1):
        r_sublistas = [list(combo) for combo in combinations(sequencia, r)]
        sublistas.extend(r_sublistas)
    return sublistas[1:]

# Testes

In [42]:
# Teste da função sequencias_binarias():

sequencias_exemplo = sequencias_binarias(3)
print(f"Sequências binárias de 3 termos: {sequencias_exemplo}")

Sequências binárias de 3 termos: [[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1], [1, 0, 0], [1, 0, 1], [1, 1, 0], [1, 1, 1]]


In [46]:
# Teste da função subsequencias():

sequencia_exemplo = [1, 2, 3]
sub_sequencias = subsequencias(sequencia_exemplo)
print(f"Sequência: {sequencia_exemplo}")
print(f"Subsequências: {sub_sequencias}")

Sequência: [1, 2, 3]
Subsequências: [[1], [2], [3], [1, 2], [1, 3], [2, 3], [1, 2, 3]]


In [44]:
# Teste da função hamiltoniano():

def C(x):
    return sum(x)

S = sequencias_binarias(3)
H = hamiltoniano(3, C)
print(H)

SparsePauliOp(['III', 'IIZ', 'IZI', 'ZII', 'IZZ', 'ZIZ', 'ZZI', 'ZZZ', 'III', 'IIZ', 'IZI', 'ZII', 'IZZ', 'ZIZ', 'ZZI', 'ZZZ', 'III', 'IIZ', 'IZI', 'ZII', 'IZZ', 'ZIZ', 'ZZI', 'ZZZ', 'III', 'IIZ', 'IZI', 'ZII', 'IZZ', 'ZIZ', 'ZZI', 'ZZZ', 'III', 'IIZ', 'IZI', 'ZII', 'IZZ', 'ZIZ', 'ZZI', 'ZZZ', 'III', 'IIZ', 'IZI', 'ZII', 'IZZ', 'ZIZ', 'ZZI', 'ZZZ', 'III', 'IIZ', 'IZI', 'ZII', 'IZZ', 'ZIZ', 'ZZI', 'ZZZ'],
              coeffs=[ 0.125+0.j, -0.125+0.j,  0.125+0.j,  0.125+0.j, -0.125+0.j, -0.125+0.j,
  0.125+0.j, -0.125+0.j,  0.125+0.j,  0.125+0.j, -0.125+0.j,  0.125+0.j,
 -0.125+0.j,  0.125+0.j, -0.125+0.j, -0.125+0.j,  0.25 +0.j, -0.25 +0.j,
 -0.25 +0.j,  0.25 +0.j,  0.25 +0.j, -0.25 +0.j, -0.25 +0.j,  0.25 +0.j,
  0.125+0.j,  0.125+0.j,  0.125+0.j, -0.125+0.j,  0.125+0.j, -0.125+0.j,
 -0.125+0.j, -0.125+0.j,  0.25 +0.j, -0.25 +0.j,  0.25 +0.j, -0.25 +0.j,
 -0.25 +0.j,  0.25 +0.j, -0.25 +0.j,  0.25 +0.j,  0.25 +0.j,  0.25 +0.j,
 -0.25 +0.j, -0.25 +0.j, -0.25 +0.j, -0.25 +0.j,  0.25 +0.j,